# Prepare Data

#### All the libraries used in the exercies

In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import sys

#### Read in the data and split into training and testing on a 0.75 split

In [2]:
data = pd.read_csv("../res/energy.txt", 
                   delimiter=",", 
                   names=['x1', 'x2', 'x3', 'x4', 'x5', 'y'])

data_train, data_test = train_test_split(data, 
                                         test_size=0.25,
                                         train_size=0.75,
                                         random_state=99,
                                         shuffle=True)

# The Loss Function

In [3]:
def loss(df: pd.DataFrame, j: str, t: float) -> float:
    x = df[j]
    n = len(x)
    left, right = get_split_mask(x, t)
    return (len(left) / n) * criteria(df, left) + \
           (len(right) / n) * criteria(df, right)


def criteria(df: pd.DataFrame, mask: list) -> float:
    if not mask:
        return 0.0
    y = df['y'].iloc[mask]
    avg = np.mean(y)
    return float(np.mean((y - avg)**2))


def get_split_mask(x: pd.Series, t: float):
    n = len(x)
    left = []
    right = []
    for i in range(n):
        if x.iloc[i] <= t:
            left.append(i)
        else:
            right.append(i)
    return left, right

## Feature/Threshold Functions

Finding the feature and threshold

* best_split - tries all combinations of features and thresholds
    * very expensive
* mean_split / median_split - uses the mean or median as a threshold along with a combination of the features

In [4]:
def best_split(df: pd.DataFrame):
    X = df.drop(['y'], axis=1)
    min_val = sys.float_info.max
    
    min_j = None
    min_t = None
    
    for _, rows in X.iterrows():
        for j, t in rows.iteritems():
            temp = loss(df, j, t)
            if temp < min_val:
                min_val = temp
                min_j = j
                min_t = t

    return min_j, min_t

In [5]:
def mean_split(df: pd.DataFrame):
    X = df.drop(['y'], axis=1)
    min_val = sys.float_info.max
    
    min_j = None
    min_t = None
    
    for j in X:
        t = np.mean(X[j])
        temp = loss(df, j, t)
        if temp < min_val:
            min_val = temp
            min_j = j
            min_t = t

    return min_j, min_t

In [6]:
def median_split(df: pd.DataFrame):
    X = df.drop(['y'], axis=1)
    min_val = sys.float_info.max
    
    min_j = None
    min_t = None
    
    for j in X:
        t = np.median(X[j])
        temp = loss(df, j, t)
        if temp < min_val:
            min_val = temp
            min_j = j
            min_t = t

    return min_j, min_t

## Testing

Just a little bit of testing for checking that the loss function works

In [20]:
test_dict = {'x1': [1, 2, 3, 4], 'x2': [3, 5, 1, 2], 'x3': [3, 4, 1, 5], 'y': [1, 3, 2, 6]}
test_df = pd.DataFrame(test_dict)

j, f = best_split(test_df)
print(j, f)
print(loss(test_df, j, f))
print(loss(test_df, 'x1', 1))
print(loss(test_df, 'x1', 2))
print(loss(test_df, 'x1', 3))
print(loss(test_df, 'x1', 4))
print(loss(test_df, 'x2', 3))
print(loss(test_df, 'x2', 5))
print(loss(test_df, 'x2', 1))
print(loss(test_df, 'x2', 2))
print(loss(test_df, 'x3', 3))
print(loss(test_df, 'x3', 4))
print(loss(test_df, 'x3', 1))
print(loss(test_df, 'x3', 5))





# assert abs(2.1969 - criteria(test_df, [0, 1, 2, 3])) < 0.001
# assert abs(criteria(test_df, [1])) < 0.000001



x3 4
0.5
2.166666666666667
2.5
0.5
3.5
3.5
3.5
3.166666666666667
2.5
1.25
0.5
3.166666666666667
3.5


In [8]:
mean_split(data_train)

('x2', 1310.9976768344673)

# Regression Tree

In [9]:
class Node:
    def __init__(self, threshold=None, feature=None, value=None,
                 left=None, right=None):
        self.threshold = threshold
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
    
    def is_leaf(self):
        return self.value is not None


class RegressionTree:
    def __init__(self, split=mean_split, max_depth=None, min_size=1):
        self.split = split
        self.max_depth = max_depth
        self.min_size = min_size
        self.root = None
    
    def fit(self, data: pd.DataFrame):
        self.root = self.build_tree(data, self.max_depth, self.min_size)
    
    def predict(self, df: pd.DataFrame):
        y_pred = pd.Series()
        for i, e in df.iterrows():
            t_root = self.root
            while not t_root.is_leaf():
                if e[t_root.feature] <= t_root.threshold:
                    t_root = t_root.left
                else:
                    t_root = t_root.right
            # y_pred.set_value(value=t_root.value, label=i)
            y_pred.at[i] = t_root.value
        return y_pred
    
    @classmethod
    def build_tree(cls, df: pd.DataFrame, max_depth=None, min_size=1):
        (r, _) = df.shape
        if r <= 0:
            raise Exception("Attempting to create an empty branch!")
        if r == min_size:
            return Node(value=float(np.mean(df['y'])))
        if max_depth is not None and max_depth == 0:
            return Node(value=float(np.mean(df['y'])))
        j, t = mean_split(df)
        left, right = get_split_mask(df[j], t)
        
        if max_depth is not None:
            max_depth -= 1
        
        return Node(left=cls.build_tree(df.iloc[left], max_depth, min_size), 
                    right=cls.build_tree(df.iloc[right], max_depth, min_size),
                    threshold=t, feature=j)


# Results

### Question 1

In [10]:
(r, _) = data_test.shape

naive = pd.Series(np.full(r, float(np.mean(data_train['y']))))

### Question 2

In [11]:
rtree = RegressionTree(max_depth=10)
rtree.fit(data_train)
y_pred = rtree.predict(data_test)


def rmse(a: pd.Series, b: pd.Series):
    return math.sqrt(mean_squared_error(a, b))

print("Naive regression: ")
print(rmse(naive, data_test['y']))

print("Mean split regression tree: ")
print(rmse(y_pred, data_test['y']))

Naive regression: 
2.72036828734702
Mean split regression tree: 
1.4393885347746098


# Citations

* Decision Trees. (2017). http://scikit-learn.org/stable/modules/tree.html#mathematical-formulation

** Note: Some of the function names from sklearn were used for clarity. The implementation of these functions are unique the solutions in this exercies.